In [1]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any
from dotenv import load_dotenv
import os

load_dotenv("./.env")

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
print(f"API Key loaded: {'Yes' if GEMINI_API_KEY else 'No'}")

API Key loaded: Yes


In [2]:
# Initialize the LLM with proper API key handling
try:
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",  # Using correct model name
        temperature=0.1,
        google_api_key=GEMINI_API_KEY
    )
    print("LLM initialized successfully")
except Exception as e:
    print(f"Error initializing LLM: {e}")
    print("Please check your GEMINI_API_KEY in the .env file")

LLM initialized successfully


In [3]:
from langchain_core.prompts import ChatPromptTemplate


In [4]:
category_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that classifies customer complaints."),
    ("user", "complaint category: delivery, refund, product issue, other."),
    ("user", "classify the following complaint: {input}")

])

response_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that generates appropriate response."),
    ("user", "Generate a professional and empathetic response to the following complaint: {complaint}. The complaint type is {complaint_type}."),
    ("user", "Note : Only single response is allowed.")
])


In [5]:
classify_chain = category_prompt | llm
response_chain = response_prompt | llm

In [6]:
from langgraph.graph import StateGraph, END
from langchain_core.output_parsers import StrOutputParser



parser = StrOutputParser()# Add output parser for cleaner responsesfrom pydantic import BaseModel, Field
from typing import Optional

In [7]:
class ComplaintState(BaseModel):
    complaint: str = Field(..., description="The customer's complaint text")
    complaint_type: str = Field(default="", description="The type of complaint")
    response: str = Field(default="", description="The response to the complaint")

In [8]:
def node_classify(state: ComplaintState) -> ComplaintState:
    # Get the AI response and extract content
    ai_response = classify_chain.invoke({"input": state.complaint})
    state.complaint_type = ai_response.content.strip().lower()
    return state

def node_respond(state: ComplaintState) -> ComplaintState:
    # Get the AI response and extract content
    ai_response = response_chain.invoke({
        "complaint": state.complaint,
        "complaint_type": state.complaint_type
    })
    state.response = ai_response.content
    return state

In [9]:
workflow = StateGraph(ComplaintState)
workflow.add_node("classify", node_classify)
workflow.add_node("respond", node_respond)

workflow.set_entry_point("classify")
workflow.add_edge("classify", "respond")
workflow.add_edge("respond", END)

In [18]:
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()
app = workflow.compile(checkpointer=checkpointer)

In [19]:
# Fix the configuration for checkpointer
config = {"configurable": {"thread_id": "complaint_thread_1"}}

state = {"complaint": "My order #12345 hasn't arrived yet and it's past the delivery date."}
result = app.invoke(state, config=config)
print(f"Complaint Type: {result['complaint_type']}")
print(f"Response: {result['response']}")

Complaint Type: delivery
Response: Dear Valued Customer,

We sincerely apologize that your order (#12345) has not yet arrived and is past its delivery date. We understand this is frustrating, and we want to assure you that we're taking this seriously.  We're currently investigating the delay and will provide you with an update within [ timeframe, e.g., 24-48 hours].  In the meantime, please could you provide us with your tracking number so we can expedite the process?  We appreciate your patience and understanding.


In [20]:
# Create a function to process complaints without memory persistence
def process_single_complaint(complaint_text: str, thread_id: str = None):
    """Process a single complaint with fresh state"""
    if thread_id is None:
        thread_id = f"thread_{hash(complaint_text) % 10000}"
    
    config = {"configurable": {"thread_id": thread_id}}
    state = {"complaint": complaint_text}
    
    try:
        result = app.invoke(state, config=config)
        return {
            "complaint": complaint_text,
            "type": result['complaint_type'],
            "response": result['response']
        }
    except Exception as e:
        return {
            "complaint": complaint_text,
            "type": "error",
            "response": f"Error: {str(e)}"
        }

# Test with multiple complaints using unique thread IDs
test_complaints = [
    "I want to return my product, it doesn't work properly",
    "Where is my refund? It's been 2 weeks since I returned the item", 
    "The delivery guy was rude to me yesterday",
    "Your website keeps crashing when I try to make a payment"
]

print("=== Testing Multiple Complaints (Clean) ===")
for i, complaint in enumerate(test_complaints):
    print(f"\n{i}. Testing: {complaint}")
    
    # Use unique thread ID for each complaint to avoid memory interference
    result = process_single_complaint(complaint, f"unique_thread_{i}_{hash(complaint)}")
    
    print(f"   Type: {result['type']}")
    print(f"   Response: {result['response'][:150]}...")
    print("-" * 80)

=== Testing Multiple Complaints (Clean) ===

0. Testing: I want to return my product, it doesn't work properly
   Type: product issue and refund
   Response: We're so sorry to hear you're experiencing trouble with your product and that it's not functioning as expected.  We understand this is frustrating, an...
--------------------------------------------------------------------------------

1. Testing: Where is my refund? It's been 2 weeks since I returned the item
   Type: refund
   Response: I understand your frustration regarding your refund; it's certainly not ideal to wait longer than expected.  We sincerely apologize for the delay.  Ou...
--------------------------------------------------------------------------------

2. Testing: The delivery guy was rude to me yesterday
   Type: delivery
   Response: We sincerely apologize that you had a negative experience with our delivery driver yesterday.  We value your business and are disappointed to hear abo...
--------------------------